```{contents}
```
## Worker Pool

A **Worker Pool** in LangGraph is a **concurrency and scalability mechanism** that allows multiple independent execution units (workers) to process graph tasks in parallel.
It transforms a single-threaded agent system into a **high-throughput, production-grade distributed execution engine**.

---

### **1. Motivation**

Large LLM workflows require:

* Parallel tool execution
* Concurrent agent reasoning
* High throughput for multiple users
* Fault isolation between tasks

A Worker Pool enables:

> **Scalable, parallel, resilient execution of LangGraph nodes.**

---

### **2. Conceptual Model**

```
Incoming Requests
        |
   Graph Scheduler
        |
  ┌───────────────┐
  │  Worker Pool  │
  └───────────────┘
   |     |     |
Worker1 Worker2 Worker3 ...
   |     |     |
 Execute Nodes in Parallel
```

Each worker is an **independent execution context** capable of running one or more graph steps.

---

### **3. Architecture Components**

| Component        | Responsibility                |
| ---------------- | ----------------------------- |
| Graph Scheduler  | Decides which node runs next  |
| Task Queue       | Holds pending node executions |
| Worker           | Executes node logic           |
| State Store      | Shared state persistence      |
| Result Collector | Merges node outputs           |
| Failure Manager  | Retries and recovery          |

---

### **4. How LangGraph Uses Worker Pools**

LangGraph’s runtime schedules node execution based on:

* Graph topology
* State readiness
* Conditional routing
* Parallel branches

Parallel branches are **dispatched to different workers** automatically.

```
Fan-out → Worker Pool → Fan-in
```

---

### **5. Example: Parallel Execution with Worker Pool**

```python
from langgraph.graph import StateGraph, END
from concurrent.futures import ThreadPoolExecutor

class State(TypedDict):
    a: int
    b: int
    c: int

def task1(state): return {"a": 1}
def task2(state): return {"b": 2}
def task3(state): return {"c": 3}

builder = StateGraph(State)
builder.add_node("t1", task1)
builder.add_node("t2", task2)
builder.add_node("t3", task3)

builder.set_entry_point("t1")
builder.add_edge("t1", "t2")
builder.add_edge("t1", "t3")
builder.add_edge("t2", END)
builder.add_edge("t3", END)

graph = builder.compile()

result = graph.invoke({}, config={"max_workers": 3})
print(result)
```

`t2` and `t3` are executed **concurrently** by different workers.

---

### **6. Worker Pool Variants**

| Variant             | Use Case              |
| ------------------- | --------------------- |
| Thread Pool         | I/O-bound tasks       |
| Process Pool        | CPU-heavy computation |
| Async Workers       | High-concurrency I/O  |
| Distributed Workers | Multi-machine scaling |

---

### **7. Production-Grade Features**

| Feature         | Description                  |
| --------------- | ---------------------------- |
| Dynamic Scaling | Adjust workers based on load |
| Backpressure    | Prevent overload             |
| Work Stealing   | Idle workers take tasks      |
| Retry Policies  | Fault recovery               |
| Timeout Control | Prevent hung workers         |
| Isolation       | Crash containment            |

---

### **8. When to Use Worker Pools**

Use worker pools when your graph:

* Has parallel branches
* Executes expensive tools
* Runs multi-agent workflows
* Serves many concurrent users
* Requires high availability

---

### **9. Performance Impact**

| Without Worker Pool   | With Worker Pool  |
| --------------------- | ----------------- |
| Sequential            | Parallel          |
| Low throughput        | High throughput   |
| High latency          | Low latency       |
| Single failure domain | Isolated failures |

---

### **10. Mental Model**

Think of LangGraph as a **distributed operating system for LLM workflows**:

* Graph = Program
* State = Memory
* Nodes = Instructions
* Worker Pool = CPU cores


### Demonstration

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import time
import random
import operator

# ---------- 1. Define Shared State with Reducer ----------
class State(TypedDict):
    results: Annotated[list, operator.add]

# ---------- 2. Define Start and Parallel Tasks ----------
def start(state: State):
    return state

def worker_a(state: State):
    time.sleep(random.uniform(0.5, 1.5))
    return {"results": ["A"]}

def worker_b(state: State):
    time.sleep(random.uniform(0.5, 1.5))
    return {"results": ["B"]}

def worker_c(state: State):
    time.sleep(random.uniform(0.5, 1.5))
    return {"results": ["C"]}

def collect(state: State):
    return {"results": [f"Collected: {', '.join(sorted(state['results']))}"]}

# ---------- 3. Build Graph ----------
builder = StateGraph(State)

builder.add_node("start", start)
builder.add_node("A", worker_a)
builder.add_node("B", worker_b)
builder.add_node("C", worker_c)
builder.add_node("collect", collect)

builder.set_entry_point("start")

# Fan-out: start branches to all workers
builder.add_edge("start", "A")
builder.add_edge("start", "B")
builder.add_edge("start", "C")

# Fan-in: all workers converge to collect
builder.add_edge("A", "collect")
builder.add_edge("B", "collect")
builder.add_edge("C", "collect")

builder.add_edge("collect", END)

graph = builder.compile()

# ---------- 4. Execute with Worker Pool ----------
start = time.time()

result = graph.invoke({"results": []}, config={"max_workers": 3})

end = time.time()

print("Final State:", result)
print("Execution Time:", round(end - start, 2), "seconds")


Final State: {'results': ['A', 'B', 'C', 'Collected: A, B, C']}
Execution Time: 1.39 seconds
